In [1]:
import pandas as pd

from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
biopsy_method = {
    '127': 'gross', '39': 'gross', '128': 'gross',
    '81': 'gross', '106': 'gross', '115': 'gross',
    '48': 'gross', '66': 'punch', '91': 'gross',
    '152': 'gross', '104': 'gross', '86': 'gross',
    '83': 'gross', '53': 'gross', '89': 'fine',
    '110': 'gross', '40': 'gross', '73': 'punch',
    '70': 'fine', '55': 'gross', '125': 'gross',
    '124': 'gross', '95': 'fine', '31': 'punch',
    '96': 'fine'
}

In [3]:
# read raw and clean TOPACIO data to isolate redacted cells
df_raw = pd.read_parquet(
    '/Users/greg/Dropbox (HMS)/topacio/cylinter_output/TOPACIO_FINAL/' +
    'output_raw/checkpoints/aggregateData.parquet'
)
df_raw['Subject'] = [i.split('_')[1].lstrip('0') for i in df_raw['Sample']]
df_raw['unique_id'] = df_raw['CellID'].astype('str') + '_' + df_raw['Subject']

df_clean = pd.read_parquet(
    '/Users/greg/Dropbox (HMS)/topacio/cylinter_output/TOPACIO_FINAL/' +
    'output_orig/checkpoints/clustering.parquet'
)
df_clean['Subject'] = [i.split('_')[1].lstrip('0') for i in df_clean['Sample']]
df_clean['unique_id'] = df_clean['CellID'].astype('str') + '_' + df_clean['Subject']

mask = df_raw['unique_id'].isin(df_clean['unique_id'])
df_dropped = df_raw[~mask]

In [4]:
# format metadata
meta = pd.read_csv(
    '/Users/greg/Dropbox (HMS)/Baker_QC_2021/nat_methods_submission/REVISION/figures_tables/'
    'online_figs/TOPACIOTiles/metadata.csv'
)
meta['Subject'] = [i.split('_')[1].lstrip('0') for i in meta['Subject']]

meta = meta[meta['Subject'].isin(biopsy_method.keys())].copy()

meta['biopsy_method'] = [biopsy_method[i] for i in meta['Subject']]

meta['BOR'] = [
    'ND' if i in ['Not Done', 'Not Evaluable']
    else 'progression' if i in ['SD', 'PD'] 
    else 'response' if i in ['PR', 'CR'] 
    else i for i in meta['Confirmed BOR']
]

meta = meta[['Subject', 'biopsy_method', 'BOR']]

In [5]:
# compute percentage of redacted cells per sample
total = df_raw.merge(meta, on='Subject', how='left')
total_counts = total.groupby(['Subject', 'biopsy_method', 'BOR']).size()

dropped_counts = df_dropped.merge(meta, on='Subject', how='left')
dropped_counts = dropped_counts.groupby(['Subject', 'biopsy_method', 'BOR']).size()

percent_dropped = (dropped_counts / total_counts) * 100

In [6]:
# run stats to compare percentage of unclustered cells with biopsy method
fine, gross, punch = percent_dropped.groupby(['biopsy_method'])

f_stat, p_value = f_oneway(fine[1].values, gross[1].values, punch[1].values)

print('Biopsy method results:')
print('Means:')
print(percent_dropped.groupby(['biopsy_method']).mean())
print('F-statistic:', f_stat)
print('P-value:', p_value)
print()

all_data = fine[1].values.tolist() + gross[1].values.tolist() + punch[1].values.tolist()
labels = (
    ['fine'] * len(fine[1].values) +
    ['gross'] * len(gross[1].values) +
    ['punch'] * len(punch[1].values)
)

tukey_results = pairwise_tukeyhsd(all_data, labels)

print(tukey_results)
print()

Biopsy method results:
Means:
biopsy_method
fine     91.916033
gross    83.086367
punch    90.778243
dtype: float64
F-statistic: 1.9266000786072355
P-value: 0.1694306961066134

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
  fine  gross  -8.8297 0.2393 -22.1196  4.4602  False
  fine  punch  -1.1378 0.9867 -19.5004 17.2249  False
 gross  punch   7.6919 0.4162  -7.3012 22.6849  False
-----------------------------------------------------



In [7]:
# run stats to compare percentage of unclustered cells with treatment response
nd, prog, resp = percent_dropped.groupby(['BOR'])

f_stat, p_value = f_oneway(nd[1].values, prog[1].values, resp[1].values)

print('Treatment response results:')
print('Means:')
print(percent_dropped.groupby(['BOR']).mean())
print('F-statistic:', f_stat)
print('P-value:', p_value)
print()

all_data = nd[1].values.tolist() + prog[1].values.tolist() + resp[1].values.tolist()
labels = (
    ['ND'] * len(nd[1].values) +
    ['progression'] * len(prog[1].values) +
    ['response'] * len(resp[1].values)
)

tukey_results = pairwise_tukeyhsd(all_data, labels)

print(tukey_results)

Treatment response results:
Means:
BOR
ND             89.064303
progression    83.507845
response       87.522855
dtype: float64
F-statistic: 0.7088144396944022
P-value: 0.5031267165404523

      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1      group2   meandiff p-adj   lower    upper  reject
---------------------------------------------------------------
         ND progression  -5.5565 0.5421 -18.6016  7.4886  False
         ND    response  -1.5414 0.9682 -17.5184 14.4355  False
progression    response    4.015  0.723  -9.0301 17.0601  False
---------------------------------------------------------------
